In [1]:
import faiss
import numpy as np
import pandas as pd
from rich.progress import track

In [2]:
df = pd.read_hdf("../data/tables/20240621/clamp_embeddings_new.h5")
df.head()

,embeddings
20231220-080-01_0000-0005_t00s00,"[0.7469334, -0.6023911, -0.5720417, 0.21939746..."
20231220-080-01_0000-0005_t00s01,"[0.4681878, -0.3210357, -0.35862225, -0.391038..."
20231220-080-01_0000-0005_t00s02,"[0.44789648, 0.028915457, -0.10429096, -0.0388..."
20231220-080-01_0000-0005_t00s03,"[0.44101596, -0.10274381, 0.0016073268, 0.1810..."
20231220-080-01_0000-0005_t00s04,"[0.67255557, -0.08258019, -0.7770954, -0.26789..."


In [4]:
def normalize_vector(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm != 0 else vec


df["normed_embeddings"] = df["embeddings"].apply(normalize_vector)
df.head()

,embeddings,normed_embeddings
20231220-080-01_0000-0005_t00s00,"[0.7469334, -0.6023911, -0.5720417, 0.21939746...","[0.048320405, -0.038969718, -0.03700636, 0.014..."
20231220-080-01_0000-0005_t00s01,"[0.4681878, -0.3210357, -0.35862225, -0.391038...","[0.032188773, -0.022071796, -0.024655938, -0.0..."
20231220-080-01_0000-0005_t00s02,"[0.44789648, 0.028915457, -0.10429096, -0.0388...","[0.030919641, 0.0019961211, -0.007199519, -0.0..."
20231220-080-01_0000-0005_t00s03,"[0.44101596, -0.10274381, 0.0016073268, 0.1810...","[0.029843057, -0.0069525586, 0.000108766006, 0..."
20231220-080-01_0000-0005_t00s04,"[0.67255557, -0.08258019, -0.7770954, -0.26789...","[0.044362742, -0.005447109, -0.05125834, -0.01..."


In [5]:
index = faiss.IndexFlatIP(768)
embedding_array = np.array(df["normed_embeddings"].tolist(), dtype=np.float32)
index.add(embedding_array)

In [6]:
k = 5
distances, indices = index.search(
    np.array(
        [df.loc["20231220-080-01_0000-0005_t00s00", "normed_embeddings"]],
        dtype=np.float32,
    ),
    k,
)

In [7]:
# Print the most similar documents
for i, index in enumerate(indices[0]):
    distance = distances[0][i]
    print(f"Nearest neighbor {i+1}: {df.iloc[index, :].name}, Distance {distance:.05f}")

Nearest neighbor 1: 20231220-080-01_0000-0005_t00s00, Distance 1.00000
Nearest neighbor 2: 20231220-080-01_0023-0029_t10s00, Distance 0.98886
Nearest neighbor 3: 20231220-080-01_0005-0011_t11s07, Distance 0.98792
Nearest neighbor 4: 20231220-080-01_0023-0029_t06s01, Distance 0.98581
Nearest neighbor 5: 20231220-080-01_0000-0005_t10s06, Distance 0.98567


In [ ]:

for i, (idx, sim) in enumerate(zip(top_indices, top_similarities)):
    print(f"Nearest neighbor {i+1}: {idx}, Distance {sim:.05f}")